In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import numpy as np
import re
import psycopg2
import random
import gc  # Garbage collector

In [2]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load model in 8-bit quantization (for RTX 4080 Super)
quant_config = BitsAndBytesConfig(load_in_8bit=True)

# Load model fully into GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Precompute token validity mask (only run once)
vocab_size = len(tokenizer.get_vocab())
all_tokens = torch.arange(vocab_size, device="cuda")  # Tensor of token IDs
decoded_tokens = tokenizer.batch_decode(all_tokens.unsqueeze(1))  # Vectorized decoding

# Create boolean mask **directly as a tensor** (avoids Python list overhead)
valid_mask = torch.tensor(
    [token.isalpha() and len(token) > 1 for token in decoded_tokens], dtype=torch.bool, device="cuda"
).clone()  # `clone()` avoids PyTorch memory issues

# Save valid token IDs & decoded token texts
allowed_tokens = torch.masked_select(all_tokens, valid_mask)  # Fast retrieval during inference
allowed_token_texts = tokenizer.batch_decode(allowed_tokens.tolist())  # Decode only once

# Preallocate masked logits tensor for reuse
masked_logits = torch.empty(vocab_size, dtype=torch.float16, device="cuda")

In [4]:
def clean_report(report: str) -> str:
    """
    Cleans the financial report by:
    - Removing excessive newlines & spaces
    - Preserving table formatting
    - Keeping important financial symbols ($, %, /)
    """
    # Preserve tables: Keep lines containing "|"
    lines = report.split("\n")
    cleaned_lines = []
    
    for line in lines:
        # Remove excessive spaces but preserve single spaces
        cleaned_line = re.sub(r"\s+", " ", line).strip()
        
        # Ensure table rows are kept intact
        if "|" in line:
            cleaned_lines.append(line)  # Keep as is
        elif cleaned_line:  # Only add meaningful lines
            cleaned_lines.append(cleaned_line)

    return "\n".join(cleaned_lines)  # Reconstruct cleaned report


In [ ]:
def split_and_tokenize_report(report: str, tokenizer, max_tokens=3800, overlap_ratio=0.2):
    """
    Splits a report into 3,800-token segments with 20% overlap and tokenizes them.
    
    Ensures tokenized outputs remain in DRAM (CPU memory) until inference.
    
    Returns:
    - A dictionary with segment indices as keys and **CPU-based** tokenized tensors as values.
    """
    inference_prompt = " Based on this financial report my investment advice is to"
    cleaned_report = clean_report(report)

    # Tokenize full report & inference prompt (ensure tensors are on CPU)
    tokens = tokenizer(cleaned_report, return_tensors="pt")["input_ids"].squeeze().to("cpu")
    prompt_tokens = tokenizer(inference_prompt, return_tensors="pt")["input_ids"].squeeze().to("cpu")

    # Define overlap
    overlap_tokens = int(max_tokens * overlap_ratio)
    
    # Initialize segments (keep them in CPU)
    token_segments = {}
    start = 0
    segment_index = 1

    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        
        # Extract segment + prompt
        segment_tokens = torch.cat((tokens[start:end], prompt_tokens), dim=0).to("cpu")  # Ensure CPU storage
        
        token_segments[f"Segment_{segment_index}"] = segment_tokens

        # Move to next segment with overlap
        start += max_tokens - overlap_tokens
        segment_index += 1

    return token_segments

In [ ]:
def fast_inference(tokens):
    """
    Performs fast inference while ensuring efficient VRAM usage.
    """
    try:
        # Move input to CUDA before inference
        inputs = {"input_ids": tokens.unsqueeze(0).to("cuda")}  # Add batch dimension

        # Get logits
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract logits for next token
        logits = outputs.logits[:, -1, :].squeeze()

        # Reset and mask logits
        masked_logits.fill_(-float("inf"))  
        masked_logits[valid_mask] = logits[valid_mask]  

        # Compute probabilities
        probs = torch.nn.functional.softmax(masked_logits, dim=-1)

        # Get corresponding probabilities for allowed tokens
        allowed_probs = torch.masked_select(probs, valid_mask)

        # Convert results to dictionary
        token_prob_dict = dict(zip(allowed_token_texts, allowed_probs.tolist()))

        return token_prob_dict

    finally:
        # Force memory cleanup
        del tokens, inputs, outputs, logits, probs, allowed_probs
        torch.cuda.empty_cache()  
        torch.cuda.ipc_collect()  # Helps free more GPU memory
        gc.collect()  # Garbage collect CPU memory



In [7]:
good_report = """
NexaTech Inc. - Q1 2024 Financial Report
For the period ended March 31, 2024

NexaTech Inc. is pleased to report strong financial results for the first quarter of 2024, driven by robust revenue growth, margin expansion, and continued execution of strategic initiatives.

Financial Performance
For the quarter ended March 31, 2024, total revenue increased 18.4% year-over-year to $1.74 billion, reflecting higher demand across core product segments and continued market penetration in key geographies. Gross profit expanded 22.7% to $764 million, driven by pricing optimization and supply chain efficiencies. Gross margin improved to 43.9%, compared to 41.6% in the prior year.

Operating income increased 27.2% to $498 million, representing an operating margin of 28.6%, up from 26.3% in Q1 2023. Net income attributable to shareholders was $382 million, a 30.1% increase over the prior-year period, translating to diluted earnings per share (EPS) of $2.74, compared to $2.08 in Q1 2023.

Strategic and Operational Highlights
During the quarter, NexaTech successfully launched its AI-driven enterprise cloud platform, achieving widespread adoption among Fortune 500 clients. The company also expanded its international presence, securing strategic partnerships in Europe and Asia-Pacific, further diversifying its revenue streams.

Capital expenditures for Q1 totaled $112 million, reflecting continued investment in AI and cloud infrastructure. The company maintained a strong balance sheet, with $2.1 billion in cash and short-term investments and a net debt-to-equity ratio of 0.24, ensuring ample liquidity to fund future growth.

Outlook
Given the strong performance in Q1 and continued market momentum, NexaTech is raising full-year 2024 guidance, expecting revenue growth of 15%–18%, with an EPS range of $10.50–$11.20, up from previous guidance of $9.80–$10.50.

The company remains committed to operational efficiency, technological innovation, and shareholder value creation, positioning itself for sustained growth in a dynamic market environment.
"""

In [8]:
bad_report = """
CoreSteel Industries - Q1 2024 Financial Report
For the period ended March 31, 2024

CoreSteel Industries reports a challenging first quarter, as macroeconomic headwinds, supply chain disruptions, and weaker-than-expected demand weighed on financial results. The company remains focused on cost management and operational efficiency while navigating ongoing market volatility.

Financial Performance
For the quarter ended March 31, 2024, revenue declined 8.7% year-over-year to $640 million, primarily due to reduced order volumes and pricing pressures in the steel manufacturing segment. Gross profit decreased 11.2% to $172 million, with gross margin contracting to 26.9%, down from 29.3% in Q1 2023, reflecting higher raw material costs.

Operating income declined 22.8% to $74 million, with operating margin falling to 11.6%, compared to 14.2% in the prior-year period. Net income attributable to shareholders was $41 million, representing a 29.4% year-over-year decrease, leading to diluted earnings per share (EPS) of $0.88, compared to $1.23 in Q1 2023.

Operational Challenges and Cost Management
CoreSteel experienced weaker demand in North America and Europe, where key customers delayed capital investments amid economic uncertainty. Additionally, higher energy and labor costs pressured margins. The company initiated a cost-reduction program targeting $50 million in annualized savings, including workforce optimization and supply chain restructuring.

Capital expenditures in Q1 were $52 million, primarily allocated to equipment upgrades and digitalization initiatives. CoreSteel ended the quarter with $284 million in cash and equivalents, maintaining financial flexibility, although net debt increased to $1.18 billion, raising leverage concerns.

Outlook
Given the uncertain economic environment, CoreSteel adjusts its full-year 2024 guidance, now anticipating revenue contraction of 4%–6%, with EPS expected between $3.00–$3.40, down from prior estimates of $3.80–$4.20. The company remains focused on cost containment, operational efficiency, and supply chain resilience while assessing opportunities for strategic realignment.

While near-term headwinds persist, CoreSteel continues to leverage its strong industry position and long-term customer relationships to drive stability and recovery.
"""


In [9]:
# Good text abd bad financials 
report_1 = """
BrightFuture Corp. - Q1 2024 Financial Report
For the period ended March 31, 2024

BrightFuture Corp. continues to demonstrate its unwavering commitment to innovation, customer success, and long-term value creation. Despite industry-wide challenges, we remain focused on strategic investments, operational excellence, and positioning ourselves for sustained growth.

Financial Performance
The first quarter of 2024 was a period of strategic transformation. While revenue came in at $520 million, a temporary 9.4% decline year-over-year, we are confident in our ability to reverse this trend in the coming quarters. Gross profit stood at $110 million, reflecting a 17.2% decrease, primarily due to short-term supply chain pressures. However, our proactive cost management initiatives are expected to drive significant improvements moving forward.

| **Metric**            | **Q1 2024** | **Q1 2023** | **YoY Change** |
|----------------------|------------|------------|--------------|
| Revenue             | $520M      | $574M      | **-9.4%**    |
| Gross Profit        | $110M      | $133M      | **-17.2%**   |
| Operating Income    | $45M       | $82M       | **-45.1%**   |
| Net Income         | $22M       | $57M       | **-61.4%**   |
| Diluted EPS        | $0.34      | $1.02      | **-66.7%**   |

Strategic Developments
Despite near-term financial headwinds, we successfully launched **three new product lines**, securing partnerships with leading industry players. Additionally, our investments in AI-driven analytics have positioned us at the forefront of the digital transformation wave.

Outlook
BrightFuture Corp. remains steadfast in delivering long-term shareholder value. With an aggressive product expansion strategy and a focus on operational efficiencies, we expect to **return to double-digit growth** in the second half of 2024.
"""


In [10]:
# Negative text and good financials 
report_2 = """
Titan Metals - Q1 2024 Financial Report
For the period ended March 31, 2024

Titan Metals faced significant macroeconomic pressures in Q1 2024, with industry volatility, rising operational costs, and an increasingly uncertain global demand landscape. Persistent inflation and labor market challenges have created a tough operating environment.

Financial Performance
Despite these headwinds, revenue surged to $1.82 billion, representing a **19.3% increase year-over-year**, supported by strong demand across industrial markets. Gross profit increased **28.7%** to $798 million, benefiting from improved pricing strategies and supply chain optimization.

| **Metric**            | **Q1 2024** | **Q1 2023** | **YoY Change** |
|----------------------|------------|------------|--------------|
| Revenue             | $1.82B     | $1.53B     | **+19.3%**   |
| Gross Profit        | $798M      | $620M      | **+28.7%**   |
| Operating Income    | $425M      | $354M      | **+20.1%**   |
| Net Income         | $320M      | $245M      | **+30.6%**   |
| Diluted EPS        | $2.96      | $2.14      | **+38.3%**   |

Operational Challenges
Despite strong financial performance, Titan Metals is facing **significant regulatory and compliance hurdles** in key global markets. Rising input costs and geopolitical risks continue to create uncertainty regarding future profitability. Additionally, competition from emerging players threatens long-term market positioning.

Outlook
Although our financials remain solid, the ongoing market volatility and cost inflation present a **serious risk to sustainable growth**. Without major industry-wide improvements, maintaining current profitability levels will be increasingly difficult in future quarters.
"""


In [11]:
# Strong text and strong financials
report_3 = """
Vertex Technologies - Q1 2024 Financial Report
For the period ended March 31, 2024

Vertex Technologies continues its trajectory of exceptional growth and innovation. Q1 2024 was marked by record-breaking financial performance, market expansion, and significant technological advancements, reinforcing our position as an industry leader.

Financial Performance
Revenue increased **22.8% year-over-year**, reaching **$2.45 billion**, driven by strong demand across all business segments. Gross profit expanded **30.1%** to **$1.12 billion**, with gross margins improving to **45.7%**.

| **Metric**            | **Q1 2024** | **Q1 2023** | **YoY Change** |
|----------------------|------------|------------|--------------|
| Revenue             | $2.45B     | $1.99B     | **+22.8%**   |
| Gross Profit        | $1.12B     | $862M      | **+30.1%**   |
| Operating Income    | $580M      | $432M      | **+34.3%**   |
| Net Income         | $450M      | $340M      | **+32.4%**   |
| Diluted EPS        | $4.18      | $3.14      | **+33.1%**   |

Strategic Developments
During the quarter, we successfully launched our **next-generation AI platform**, further enhancing our competitive advantage. Expansion into **Europe and Latin America** has unlocked significant revenue potential, reinforcing our commitment to global growth.

Outlook
With continued momentum, we are **raising our full-year 2024 guidance**, anticipating **20-25% revenue growth** and **expanding margins**. Our unwavering focus on innovation and market leadership positions us for sustained success.
"""


In [12]:
# Poor text and poor financials
report_4 = """
Delta Industrial Solutions - Q1 2024 Financial Report
For the period ended March 31, 2024

Delta Industrial Solutions faced a particularly difficult first quarter, as ongoing supply chain disruptions, declining order volumes, and cost pressures weighed heavily on performance. The company continues to navigate **serious operational inefficiencies** that have significantly impacted margins.

Financial Performance
Revenue declined **11.6%** year-over-year to **$415 million**, with gross profit falling **19.2%** due to rising material costs and lower-than-expected sales.

| **Metric**            | **Q1 2024** | **Q1 2023** | **YoY Change** |
|----------------------|------------|------------|--------------|
| Revenue             | $415M      | $470M      | **-11.6%**   |
| Gross Profit        | $94M       | $116M      | **-19.2%**   |
| Operating Income    | $30M       | $65M       | **-53.8%**   |
| Net Income         | $14M       | $42M       | **-66.7%**   |
| Diluted EPS        | $0.28      | $0.94      | **-70.2%**   |

Operational Challenges
Persistent **inventory shortages**, **higher freight costs**, and **weakened customer demand** have led to **substantial revenue losses**. Cost-saving initiatives have been implemented, but the near-term outlook remains **highly uncertain**.

Outlook
Given the sustained decline in key financial metrics, Delta Industrial Solutions **anticipates further contraction** in Q2 2024. The company is currently evaluating **strategic restructuring options** to improve long-term viability.
"""


In [13]:
def get_random_report():
    """
    Fetches a random financial report from the database.
    """
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname="reports_db",
        user="postgres",     # Change to your actual username
        password="postgres", # Change to your actual password
        host="localhost",         # Change if needed
        port="5432"               # Default PostgreSQL port
    )
    
    try:
        with conn.cursor() as cur:
            # Count total reports
            cur.execute("SELECT COUNT(*) FROM reports;")
            total_reports = cur.fetchone()[0]
            
            if total_reports == 0:
                print("No reports available in the database.")
                return None
            
            # Select a random offset
            random_offset = random.randint(0, total_reports - 1)
            
            # Fetch a random report
            cur.execute("""
                SELECT raw_text FROM reports 
                OFFSET %s LIMIT 1;
            """, (random_offset,))
            
            report = cur.fetchone()[0]
            return report  # Return the raw financial report text
            
    finally:
        conn.close()

random_report = get_random_report()
if random_report:
    print("Fetched report:\n", random_report[:500])  # Print the first 500 characters

Fetched report:
 Table of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549Form 10-Q(Mark One)QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the quarterly period ended June 30, 2020 ORTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the transition period from              to             Commission File Number: 001-33784SANDRIDGE ENERGY, INC.(Exact name of registrant as specified in its charter)Delaw


In [14]:
def create_verbolizer(positive_words: list, negative_words: list) -> dict:
    positive_words = list(map(str.lower, positive_words))
    negative_words = list(map(str.lower, negative_words))
                          
    verbolizer = {
        "positive" : [word.capitalize() for word in positive_words] + positive_words,
        "negative" : [word.capitalize() for word in negative_words] + negative_words,
    }

    return verbolizer

In [15]:
verbolizer = create_verbolizer(
        positive_words=[
            "buy", "invest", "purchase", "Invest", "buying", "stay",
            "proceed", "recommend", "Hold", "retain", "increase",
            "maintain", "acquire"
        ],
        negative_words=[
            "sell", "avoid", "caut", "carefully", "closely", "caution",
            "analyze", "minimize", "Avoid", "decrease", "Wait",
            "investigate", "sold", "decline", "Monitor", "assess",
            "sale", "remove", "seriously"
        ],
        )

In [16]:
def get_score(token_prob_dict: dict, verbolizer: dict) -> float:
  
    positive_prob = sum(token_prob_dict.get(word, 0) for word in verbolizer["positive"])
    negative_prob = sum(token_prob_dict.get(word, 0) for word in verbolizer["negative"])

    return positive_prob - negative_prob
        
    

In [ ]:
token_prob_dict = fast_inference(random_report)
# Step 7: Sort and display top predictions
sorted_token_probs = {k: v for k, v in sorted(token_prob_dict.items(), key=lambda item: item[1], reverse=True)}

print("\n🔹 **Next Token Prediction Probabilities (Only Meaningful Words):**")
for token, prob in list(sorted_token_probs.items())[:100]:  
    print(f"{token:<10} | Probability: {prob:.4f}")

In [18]:
get_score(token_prob_dict, verbolizer)

NameError: name 'token_prob_dict' is not defined

In [ ]:
def compute_sample_scores():
    rand_rep = get_random_report()  # Fetch a random report
    clean_rep = clean_report(rand_rep)  # Clean the report
    tokenized_segments = split_and_tokenize_report(clean_rep, tokenizer)  # Tokenize & split

    for segment_name, tokens in tokenized_segments.items():
        print(f"Processing {segment_name}...")

        # Track VRAM before inference
        before_mem = torch.cuda.memory_allocated()
        
        # Move tokens to CUDA before inference
        tokens = tokens.to("cuda")

        # Run inference
        token_prob_dict = fast_inference(tokens)

        # Compute and print score
        print(get_score(token_prob_dict, verbolizer))

        # Delete tokens immediately to free VRAM
        del tokens  # Remove reference
        torch.cuda.empty_cache()  # Free memory
        torch.cuda.ipc_collect()  # Helps reclaim GPU memory
        gc.collect()  # Clean up CPU-side memory

compute_sample_scores()


Processing Segment_1...
0.0721403956413269
Processing Segment_2...
0.257865846157074
Processing Segment_3...
0.10415458679199219
Processing Segment_4...
0.03744298219680786
Processing Segment_5...
0.22798848152160645
Processing Segment_6...
0.4859278202056885
Processing Segment_7...
-0.114696204662323
Processing Segment_8...
0.2649211287498474
Processing Segment_9...
0.3060307502746582


In [ ]:
def get_prompt(text: str) -> str:
    #appendics = "Based on this financial report my investment advice is to"
    appendics = "How do you think, Is it a real financial report or a fake one and why? Explain in details. I think"
    return text + appendics

In [ ]:
prompt = get_prompt(report_4)

inputs = tokenizer(prompt,return_tensors="pt").to("cuda") 

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=150)

generated_tokens = outputs[0, inputs.input_ids.shape[1]:]

text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
text

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"it is a fake one because it is not consistent with the company's historical financial performance. The company's revenue has been consistently increasing over the past few years, and the gross profit margin has been consistently improving. Therefore, it is unlikely that the company would experience a significant decline in revenue and gross profit in Q1 2024. Additionally, the company's net income has been consistently positive over the past few years, and the diluted EPS has been consistently increasing. Therefore, it is unlikely that the company would experience a significant decline in net income and diluted EPS in Q1 2024. Furthermore, the company's cost-saving initiatives have been consistently successful in the past"